<a href="https://colab.research.google.com/github/harshalDharpure/Multimodality_Hateful_Meme/blob/main/baseline_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Modified for Hindi datasets Prompthate code

In [ ]:
import torch
import torch.nn as nn
from transformers import RobertaForMaskedLM

class RobertaPromptModel(nn.Module):
    def __init__(self, label_list):
        super(RobertaPromptModel, self).__init__()
        self.label_word_list = label_list
        self.roberta = RobertaForMaskedLM.from_pretrained('roberta-large')

    def forward(self, tokens, attention_mask, mask_pos, feat=None):
        batch_size = tokens.size(0)
        # The position of the word for prediction
        if mask_pos is not None:
            mask_pos = mask_pos.squeeze()

        out = self.roberta(input_ids=tokens, attention_mask=attention_mask)
        prediction_mask_scores = out.logits[torch.arange(batch_size), mask_pos]

        logits = []
        for label_id in range(len(self.label_word_list)):
            logits.append(prediction_mask_scores[:, self.label_word_list[label_id]].unsqueeze(-1))
        logits = torch.cat(logits, -1)
        return logits


def build_baseline(opt, label_list):
    return RobertaPromptModel(label_list)
